# Agentpy Example

## Config

In [119]:
#! pip install agentpy matplotlib seaborn IPython

## Source code

#### Imports

In [120]:
# Model design
import agentpy as ap

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import IPython
import random


#### Model definition

In [121]:
MOVES = [(1,0),(0,1),(-1,0),(0,-1),(0,0),(1,1),(-1,1),(1,-1)]

class RoomModel(ap.Model):

    def setup(self):

        self.robots = ap.AgentList(self,int(self.p['robot']))
        self.trash = ap.AgentList(self,int(self.p['density']* (self.p.N * self.p.M)))
        self.moves = 0

        self.room = ap.Grid(self, [self.p.N,self.p.M], track_empty = True)
        self.room.add_agents(self.robots,[(1,1)]*self.p['robot'])
        self.room.add_agents(self.trash,random = True)

        # Initiate a dynamic variable for all trees
        # Condition 0: robot, 1: trash, 2: clean
        self.robots.type_agent = 0
        self.trash.type_agent = 1

    def step(self):

        # Select robot
        robots = self.robots
        
        for robot in robots:
            for neighbor in self.room.neighbors(robot):
                if neighbor.type_agent == 1:
                    neighbor.type_agent = 2
                    break
            else:
                self.room.move_by(robot, random.choice(MOVES))
                self.moves += 1

        trash_pending = self.trash.select(self.trash.type_agent == 1)

        # Stop simulation if no fire is left
        if len(trash_pending) == 0:
            self.stop()

    def end(self):

        # Document a measure at the end of the simulation
        cleaned_tiles = len(self.trash.select(self.trash.type_agent == 2))
        self.report('Percentage of cleaned tiles',
                    cleaned_tiles / len(self.trash))

In [122]:
parameters = {
    'density':0.3,
    'N':20,
    'M':20,
    'steps':100,
    'robot':10,
}

#### Create single-run animation with custom colors

In [123]:
def animation_plot(model, ax):
    initial_trash = int(model.p['density'] * (model.p.N * model.p.M))
    cleaned = float("{:.2f}".format((len(model.trash.select(model.trash.type_agent == 2)) / initial_trash) *100))

    attr_grid = model.room.attr_grid('type_agent')
    color_dict = {0:'#0325FF', 1:'#762100', 2:'#e5e5e5', None:'#d5e5d5'}
    ap.gridplot(attr_grid, ax=ax, color_dict=color_dict, convert=True)
    ax.set_title(f"Simulation of Cleaning robots\n"
                 f"Time-step: {model.t}, Trash left: "
                 f"{len(model.trash.select(model.trash.type_agent == 1))}\n"
                 f"Cleaned trash: "
                 f"{cleaned}%"
                 f"Total moves: "
                 f"{model.moves}")

fig, ax = plt.subplots()
model = RoomModel(parameters)
animation = ap.animate(model, fig, ax, animation_plot)
IPython.display.HTML(animation.to_jshtml(fps=15))